### activation is int8, weights is per channel wise quant


In [2]:
import torch
from supports.quant_base_method import quant_per_tensor_asymmetric_int8_with_val

input_tensor = torch.rand(2, 3, 4, dtype=torch.float32).uniform_(-1.0, 3.0)

# asymmetric quantization
asym_input, input_scales, input_zp = quant_per_tensor_asymmetric_int8_with_val(input_tensor)

tensor(-43, dtype=torch.int8)


In [3]:
from supports.quant_base_method import channel_wise_symmetric_quantize

# per channel quantization weights
weights_tensor = torch.rand(5, 4, dtype=torch.float32).uniform_(-2.0, 2.0)
print(weights_tensor)
bias_tensor = torch.rand(5, dtype=torch.float32).uniform_(-2.0, 2.0)
print(bias_tensor)

output_f32 = torch.matmul(input_tensor, weights_tensor.t()) + bias_tensor
q_output_f32, output_scales, output_zp = quant_per_tensor_asymmetric_int8_with_val(output_f32, max_val=10, min_val=-10)
print(q_output_f32)

tensor([[-0.5876,  1.8591,  1.1977, -0.6384],
        [ 1.1254,  0.2665, -1.4759,  0.7779],
        [-0.2259, -1.6501,  1.0892, -1.7259],
        [-0.4059,  0.4066,  0.3698, -1.9416],
        [ 0.2653, -0.3159, -0.4189, -1.3381]])
tensor([ 1.9125, -1.6158,  0.9849,  0.7302, -0.3656])
tensor(0, dtype=torch.int8)
tensor([[[ 26,  37, -96, -60, -54],
         [ 31, -34,  28,   8, -10],
         [127, -56, -11,  31, -32]],

        [[ 40, -40,  35,  26,   1],
         [ 14,  28, -53, -60, -47],
         [ 10,  10, -57, -51, -48]]], dtype=torch.int8)


In [4]:
per_channel_monitor, channel_wise = channel_wise_symmetric_quantize(weights_tensor.t())
print(per_channel_monitor)
print(channel_wise)

tensor([[ -40,   97,  -17,  -27,   25],
        [ 127,   23, -121,   27,  -30],
        [  82, -127,   80,   24,  -40],
        [ -44,   67, -127, -127, -127]], dtype=torch.int8)
tensor([0.0146, 0.0116, 0.0136, 0.0153, 0.0105])


In [5]:
from quant_ops.quant_linear import W8A8B32O8LinearForMac

origin_linear: torch.nn.Module = torch.nn.Linear(4, 5)
# 1. 直接修改权重的值
origin_linear.weight.data.copy_(weights_tensor)
origin_linear.bias.data.copy_(bias_tensor)

ops: W8A8B32O8LinearForMac = W8A8B32O8LinearForMac.from_float(module=origin_linear,
                                                              input_scale=input_scales,
                                                              input_zp=input_zp,
                                                              output_scale=output_scales,
                                                              output_zp=output_zp)

q_output, o_output = ops(asym_input)
print(q_output)
print(o_output)


tensor([[[  9,  50, -89, -48, -35],
         [  7, -12,  15,   0,  -4],
         [ 73, -22, -18,  14, -19]],

        [[ 16, -18,  23,  17,   5],
         [  0,  32, -51, -51, -35],
         [ -9,  25, -58, -47, -34]]], dtype=torch.int8)
tensor([[[  4103,  28714, -44082, -20956, -22153],
         [  3360,  -7406,   7781,   -195,  -3057],
         [ 33413, -12803,  -8988,   6212, -12325]],

        [[  7506, -10756,  11447,   7525,   3513],
         [   191,  18718, -25053, -22465, -22548],
         [ -4381,  14851, -28734, -20740, -21983]]], dtype=torch.int32)


In [6]:
print(output_f32)

tensor([[[ 2.0843,  2.9468, -7.4702, -4.6374, -4.1752],
         [ 2.4841, -2.6226,  2.2167,  0.6921, -0.7242],
         [10.0681, -4.3828, -0.8257,  2.4475, -2.4599]],

        [[ 3.1800, -3.0604,  2.7773,  2.0580,  0.0826],
         [ 1.1247,  2.2514, -4.0932, -4.6438, -3.6131],
         [ 0.8495,  0.7875, -4.4329, -3.9332, -3.7006]]])


In [10]:
from supports.quant_base_method import quant_bias
q_bias = quant_bias(bias_tensor, input_scales, weight_scale=channel_wise)
output_s32 = (torch.matmul((asym_input.to(torch.int32) - input_zp), per_channel_monitor.to(torch.int32)) + q_bias)
print(output_s32)

tensor([[[  4103,  28714, -44082, -20956, -22153],
         [  3360,  -7406,   7781,   -195,  -3057],
         [ 33413, -12803,  -8988,   6212, -12325]],

        [[  7506, -10756,  11447,   7525,   3513],
         [   191,  18718, -25053, -22465, -22548],
         [ -4381,  14851, -28734, -20740, -21983]]], dtype=torch.int32)
